# SPR 2026 - LinearSVC v4 (Calibration + Platt Scaling)

**Score baseline:** 0.77885
**Score v3:** 0.75966 (regrediu -2.5%)

**Estratégia v4:** Calibração de probabilidades
- Platt Scaling para calibrar as probabilidades
- Isotonic Regression como alternativa
- Threshold tuning após calibração
- NÃO usar RandomSearch (que quebrou no v3)

**Meta:** Recuperar 0.77+ e possivelmente melhorar

---
**CONFIGURAÇÃO KAGGLE:** Internet OFF
---

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - LinearSVC v4 (Calibration + Platt Scaling)")
print("="*60)

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# ========== VERIFICAR DATASET PRIMEIRO ==========
if not os.path.exists(DATA_DIR):
    print("\n" + "="*60)
    print("ERRO: Dataset não encontrado!")
    print("="*60)
    print("\nAdicione o dataset:")
    print("Add Input → Competition → spr-2026-mammography-report-classification")
    raise FileNotFoundError(f"Dataset não encontrado: {DATA_DIR}")
print(f"Dataset: {DATA_DIR}")

np.random.seed(SEED)

# ========== CONFIGURAÇÕES v4 ==========
USE_PLATT_SCALING = True        # Platt scaling (sigmoid)
USE_ISOTONIC = False            # Alternativa: isotonic regression
USE_THRESHOLD_TUNING = True     # Threshold após calibração
CV_FOLDS = 5                    # Folds para calibração
# ======================================

# Dados
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {train.shape} | Test: {test.shape}")

# Auto-detectar colunas
def find_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
        for col in df.columns:
            if col.lower() == c.lower():
                return col
    return None

TEXT_COL = find_col(train, ['report', 'text', 'laudo', 'texto', 'content'])
LABEL_COL = find_col(train, ['target', 'label', 'birads', 'classe', 'class'])
ID_COL = find_col(test, ['ID', 'id', 'Id', 'index', 'idx'])
print(f"Colunas: texto={TEXT_COL}, label={LABEL_COL}, id={ID_COL}")

In [ ]:
# TF-IDF (config original que funcionou)
print("\n[2/5] TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=15000, 
    ngram_range=(1, 2), 
    min_df=2, 
    max_df=0.95, 
    sublinear_tf=True
)
X_train = tfidf.fit_transform(train[TEXT_COL])
X_test = tfidf.transform(test[TEXT_COL])
y_train = train[LABEL_COL].values
print(f"Shape: {X_train.shape}")

In [ ]:
# LinearSVC base com configuração original
print("\n[3/5] LinearSVC + Calibração...")

# Modelo base (config original que funcionou)
base_svc = LinearSVC(
    C=1.0,                      # Config original
    class_weight='balanced',
    random_state=SEED,
    max_iter=5000,
    tol=1e-4,
    dual=True
)

# Calibração com Platt Scaling ou Isotonic
calibration_method = 'sigmoid' if USE_PLATT_SCALING else 'isotonic'
print(f"Método de calibração: {calibration_method}")

calibrated_svc = CalibratedClassifierCV(
    base_svc, 
    cv=CV_FOLDS, 
    method=calibration_method
)

print("Treinando modelo com calibração...")
calibrated_svc.fit(X_train, y_train)
print("Modelo treinado!")

# Cross-validation score
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
cv_scores = cross_val_score(calibrated_svc, X_train, y_train, cv=cv, scoring='f1_macro')
print(f"CV F1-macro: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

In [ ]:
# Inferência com threshold tuning
print("\n[4/5] Inferência...")

proba = calibrated_svc.predict_proba(X_test)
classes = calibrated_svc.classes_

print(f"Probabilidades shape: {proba.shape}")
print(f"Classes: {classes}")

if USE_THRESHOLD_TUNING:
    print("\nAplicando threshold tuning...")
    
    thresholds = {
        0: 0.50,
        1: 0.50,
        2: 0.50,
        3: 0.50,
        4: 0.50,
        5: 0.35,   # Classe minoritária
        6: 0.30,   # Classe muito minoritária
    }
    
    print("Thresholds:")
    for c, t in thresholds.items():
        print(f"  Classe {c}: {t}")
    
    predictions = []
    for i in range(len(proba)):
        adj = proba[i].copy()
        for j, c in enumerate(classes):
            if c in thresholds:
                adj[j] *= (0.5 / thresholds[c])
        predictions.append(classes[np.argmax(adj)])
    predictions = np.array(predictions)
else:
    predictions = calibrated_svc.predict(X_test)

In [ ]:
# Submissão
print("\n[5/5] Gerando submissão...")

sample_path = f'{DATA_DIR}/sample_submission.csv'
if os.path.exists(sample_path):
    sample_sub = pd.read_csv(sample_path)
    SUB_ID = sample_sub.columns[0]
    SUB_LABEL = sample_sub.columns[1]
else:
    SUB_ID = ID_COL
    SUB_LABEL = LABEL_COL

submission = pd.DataFrame({SUB_ID: test[ID_COL], SUB_LABEL: predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("="*60)
print("LinearSVC v4 CONCLUÍDO!")
print("="*60)
print(f"\nCV F1-macro: {cv_scores.mean():.4f}")
print(f"Calibração: {calibration_method}")
print(f"Threshold tuning: {'ATIVO' if USE_THRESHOLD_TUNING else 'DESATIVADO'}")
print("\nDistribuição das predições:")
print(submission[SUB_LABEL].value_counts().sort_index())
print("\n✅ submission.csv criado!")